# River Tree

An example workflow for gathering NHD Plus data and generating a tree.

AJ: works with ats_meshing_dev conda environment

In [1]:
import os,sys
sys.path.append('/Users/ajc/Core/SimDataInputs/WorkSpace/ats-88/ats_meshing/workflow/')
import numpy as np
from matplotlib import pyplot as plt
import shapely
import logging
sys.path.append('/Users/ajc/anaconda3/envs/ats_meshing/')
import workflow
import workflow.source_list
import workflow.conf
import workflow.ui
import workflow.utils
import workflow.plot
workflow.conf.rcParams['data dir'] = os.path.join(os.getcwd(),'..','data')

workflow.ui.setup_logging(1,None)

In [2]:
# open a shapefile for use here
#use fiona to read shape file
manager_shp = workflow.source_list.FileManagerShape('../data/hydrologic_units/others/coweeta_basin/coweeta_basin.shp')
shp_profile,shp = manager_shp.get_shape()
shply = workflow.utils.shply(shp)

/Users/ajc/anaconda3/envs/ats_meshing_dev/lib/python3.7/site-packages/fiona/collection.py:336: FionaDeprecationWarning: Collection slicing is deprecated and will be disabled in a future version.
  return self.session.__getitem__(item)


In [3]:
# find the rivers in this shape
#call to hilev.py
#reaches, _ = workflow.get_rivers_by_bounds(workflow.source_list.hydrography_sources['NHD Plus'],
#                                               shply.bounds, shp_profile['crs'], '0601', merge=False)

crs, reaches = workflow.get_reaches(workflow.source_list.hydrography_sources['NHD Plus'], '0601',
                                               shply.bounds, shp_profile['crs'], long=None, merge=False)

# filter the list to only those that intersect the shape
reaches = workflow.hydrography.filter_rivers_to_shape(shply, reaches, 10)

# make the global tree
rivers = workflow.hydrography.make_global_tree(reaches)

# check that only one tree was formed (this means al)
assert(len(rivers) is 1)
river = rivers[0]
print (river)

2020-12-08 18:06:45,244 - root - INFO: 
2020-12-08 18:06:45,245 - root - INFO: Preprocessing Hydrography
2020-12-08 18:06:45,245 - root - INFO: ------------------------------
2020-12-08 18:06:45,246 - root - INFO: loading streams in HUC 0601
2020-12-08 18:06:45,247 - root - INFO: and/or bounds (273971.0911428096, 3878839.6361173145, 279140.9150949494, 3883953.7853134344)
2020-12-08 18:06:45,277 - root - INFO: Using Hydrography file "/Users/ajc/Core/SimDataInputs/WorkSpace/ats-88/ats_meshing/examples/../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb"
2020-12-08 18:06:51,709 - root - INFO:   ...filtering


In [4]:
"""
%matplotlib
HSV_tuples=['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', 
            '#fabebe', '#008080', '#e6beff', '#9a6324', '#a9a9a9', '#800000', '#aaffc3', '#808000', '#ffd8b1', 
            '#000075', '#808080', '#000000', '#009090', '#090770']
workflow.plot.rivers([river,], shp_profile['crs'],color=HSV_tuples)
"""


"\n%matplotlib\nHSV_tuples=['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', \n            '#fabebe', '#008080', '#e6beff', '#9a6324', '#a9a9a9', '#800000', '#aaffc3', '#808000', '#ffd8b1', \n            '#000075', '#808080', '#000000', '#009090', '#090770']\nworkflow.plot.rivers([river,], shp_profile['crs'],color=HSV_tuples)\n"

In [5]:
#import colorsys
#N = 22
#HSV_tuples = [(x*1.0/N, x*.1/N, 1.0) for x in range(N)]
#RGB_tuples = map(lambda x: colorsys.hsv_to_rgb(*x), HSV_tuples)

In [6]:
river_NHDPlusID = []
for r in river:
    #print (r,r.properties)
    river_NHDPlusID.append(r.properties['NHDPlusID'])    

print (river_NHDPlusID, len(river_NHDPlusID))

[25000400040729.0, 25000400108020.0, 25000400040724.0, 25000400007107.0, 25000400007106.0, 25000400175089.0, 25000400074075.0, 25000400107798.0, 25000400108019.0, 25000400175304.0, 25000400040725.0, 25000400007320.0, 25000400074306.0, 25000400007109.0, 25000400074076.0, 25000400007108.0, 25000400040325.0, 25000400141373.0, 25000400175091.0, 25000400175090.0, 25000400208871.0, 25000400242778.0] 22


In [7]:
import fiona
fiona.listlayers('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb')

['ExternalCrosswalk',
 'FeatureToMetadata',
 'HUMod',
 'NHDFCode',
 'NHDFeatureToMetadata',
 'NHDMetadata',
 'NHDPlusDivFracMP',
 'NHDPlusEROMMA',
 'NHDPlusEROMQAMA',
 'NHDPlusEROMQARPT',
 'NHDPlusFlow',
 'NHDPlusFlowlineVAA',
 'NHDPlusIncrLat',
 'NHDPlusIncrPrecipMA',
 'NHDPlusIncrPrecipMM01',
 'NHDPlusIncrPrecipMM02',
 'NHDPlusIncrPrecipMM03',
 'NHDPlusIncrPrecipMM04',
 'NHDPlusIncrPrecipMM05',
 'NHDPlusIncrPrecipMM06',
 'NHDPlusIncrPrecipMM07',
 'NHDPlusIncrPrecipMM08',
 'NHDPlusIncrPrecipMM09',
 'NHDPlusIncrPrecipMM10',
 'NHDPlusIncrPrecipMM11',
 'NHDPlusIncrPrecipMM12',
 'NHDPlusIncrROMA',
 'NHDPlusIncrTempMA',
 'NHDPlusIncrTempMM01',
 'NHDPlusIncrTempMM02',
 'NHDPlusIncrTempMM03',
 'NHDPlusIncrTempMM04',
 'NHDPlusIncrTempMM05',
 'NHDPlusIncrTempMM06',
 'NHDPlusIncrTempMM07',
 'NHDPlusIncrTempMM08',
 'NHDPlusIncrTempMM09',
 'NHDPlusIncrTempMM10',
 'NHDPlusIncrTempMM11',
 'NHDPlusIncrTempMM12',
 'NHDPlusMegaDiv',
 'NHDPlusNHDPlusIDGridCode',
 'NHDProcessingParameters',
 'NHDReachCo

In [8]:
FlowlineVAA = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDPlusFlowlineVAA')
Flowline = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDFlowline')
Flow = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDPlusFlow')
Line = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDLine')
NHDArea = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDArea')

FlowlineVAA[1]['properties']['TotDASqKm']
#NHDArea[1]
#Flow[1]

0.14690001

In [9]:
NHDPLusEROMMA = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDPlusEROMMA')
NHDPLusEROMQAMA = fiona.open('../data/hydrography/NHDPlus_H_0601_GDB/NHDPlus_H_0601.gdb', 'r', layer='NHDPlusEROMQAMA')
NHDPLusEROMMA[1]

{'type': 'Feature',
 'id': '1',
 'properties': OrderedDict([('NHDPlusID', 25000400123019.0),
              ('QAMA', 0.059011999999999995),
              ('VAMA', 0.62450043),
              ('QIncrAMA', 0.00195),
              ('QBMA', 0.059011999999999995),
              ('VBMA', 0.62450043),
              ('QIncrBMA', 0.00195),
              ('QCMA', 0.059011999999999995),
              ('VCMA', 0.62450043),
              ('QIncrCMA', 0.00195),
              ('QDMA', 0.059011999999999995),
              ('VDMA', 0.62450043),
              ('QIncrDMA', 0.00195),
              ('QEMA', 0.059011999999999995),
              ('VEMA', 0.62450043),
              ('QIncrEMA', 0.00195),
              ('QFMA', 0.059011999999999995),
              ('QIncrFMA', 0.00195),
              ('ArQNavMA', 0.0),
              ('PETMA', 0.0),
              ('QLossMA', 0.0),
              ('QGAdjMA', 0.0),
              ('QGNavMA', 0.0),
              ('GageAdjMA', 0),
              ('AvgQAdjMA', 0.0),
    

In [10]:
import math

f_vector = lambda p,q: [q[0] - p[0], q[1] - p[1],  q[2] - p[2]]
f_distance = lambda p,q: np.sqrt ((p[0] - q[0])**2 + (p[1] - q[1])**2 + (p[2] - q[2])**2)

f_normal = lambda v,d: [v[0]/d, v[1]/d, v[2]/d]

def get_normal(coords):
    pos_v = f_vector(coords[0], coords[1])
    distance = math.sqrt(pos_v[0]**2 + pos_v[1]**2 + pos_v[2]**2)
    normal = f_normal(pos_v, distance)
    normal = [round(n,6) for n in normal]
    return normal

In [11]:
km_to_m = 1000.
def River_stream_length():
    d2 = []
    for r in river:
        d1 = [('NHDPlusID',r.properties['NHDPlusID']), 
              ('LengthKM',r.properties['LengthKM']*km_to_m)]
        d2.append(d1)
    
    d3 = dict()
    d3['stream_lengthKM'] = d2
    return d3

In [12]:
DataLength = River_stream_length()

In [13]:
#QAMA : Flow from runoff, MA: Mean Annual
#VANA : Velocity for QAMA
#TotDASqKm: TotalDrainageAreaSqKm [may not be what we want??]

def rating_curve(q_cfs,v_fs):
    b, f, m = 0.26, 0.40, 0.34
    w = x1 * q_cfs**b
    d = x2 * q_cfs**f
    v = x3 * q_cfs**m
    
import scipy.signal
import geopy.distance


def cell_lengths_and_centroids_simple(index,x_spacing=-1,outlet_cell=False,spacing='Uniform'):
    length = np.round(Flowline[index]['properties']['LengthKM']*1000,4)
    xx = Flowline[index]['properties']['NHDPlusID']
    
    if (spacing == "Uniform"):
        if (x_spacing == -1 or length<=x_spacing):
            d = 1
        else:
            d = int(length/x_spacing) + 1
            
        dx = length/d # ensures all dx sum to the total length
        spacing = np.round(np.arange(0,length+dx,dx),4)
        cells_spacing = [c for c in spacing if c<=length]
        cells_length = cells_spacing[1:] # first cell entity is ignored, zero length
        
    ncells = len(cells_length) if len(cells_length) > 0 else 1
    
    sum1 = 0
    #print ('Length and spacing: ', cells_spacing[-1], length)
    assert (cells_spacing[-1] == length)
    #print ('Ncells:', ncells, cells_spacing, length)
    x1 = Flowline[index]['geometry']['coordinates'][0]
    ncoords = len(x1)
    
    
    c1 = int(ncoords/(ncells))
    remain = ncoords%ncells # For Vis only: "remain" cells have large volume, nope??
    p_index = []
    
    for i in range(ncells+1):
        #print ('Remain: ',i, remain,c1)
        if (remain == 0 and ncoords >= ncells):
            p_index.append(i*(c1-1))
        elif (remain == 0 and ncoords < ncells):
            p_index.append(c1-1)
        elif i < remain:
            #print (i)
            p_index.append(i*(c1+1)) 
        else:
            p_index.append(p_index[i-1] + c1)
    
    edges_coords = []
    for p in p_index:
        coord = Flowline[index]['geometry']['coordinates'][0][p]
        edges_coords.append(coord)
        
        
    centroids = []
    #print ('Edges: ', len(edges_coords))
    if (x_spacing >0):
        for i in range(len(edges_coords)-1,0,-1):
            cent = [0.5* (edges_coords[i-1][0] + edges_coords[i][0]), 0.5* (edges_coords[i-1][1] + edges_coords[i][1]), 0]
            centroids.append(cent)
    else:
        # this is for base-case in which the stream length is the cell size
        i = 1
        cent = [0.5* (edges_coords[i-1][0] + edges_coords[i][0]), 0.5* (edges_coords[i-1][1] + edges_coords[i][1]), 0]

        centroids.append(cent)
    
    cells_spacing = list(cells_spacing)
    cells_length1 = [cells_spacing[1],]*len(cells_length)
    
    if (outlet_cell):
        centroids.insert(0, [edges_coords[-1][0] - 0.001,edges_coords[-1][1] + 0.001,0])
        cells_spacing = np.insert(cells_spacing,1,25.)
        cells_spacing[2] = cells_spacing[2] - 25.
        cells_length1 = np.insert(cells_length1,0,25)
        cells_length1[1] = cells_length1[1] - 25.
    
    centroids = list(np.concatenate(centroids))

    assert (len(centroids) == 3*len(cells_length1))
    #print ('L: ', cells_length1, cells_spacing, len(cells_length1),np.sum(cells_length1))
    return centroids, cells_length1, cells_spacing, len(cells_length1)

In [14]:
DataFlow = dict()
discharge = []
velocity = []

Normal = []
segment_depth = []
segment_length = []
segment_width = []
segment_crossA = []

km_to_m = 1000.
cfs_to_ms = 0.028316847 # QAMA is in cfs, convert cfs to m/s
fs_to_ms = 0.3048

# get discharge, velocity, depth, and orientation

def River_Characteristics(x_spacing_=-1.):
    d4 = []
    d5 = []
    d6 = []
    add_outlet_cell = True
    if x_spacing_ <=25.:
        add_outlet_cell = False
    for k,r in enumerate(river):
        for i in range(len(NHDPLusEROMMA)):
            if NHDPLusEROMMA[i+1]['properties']['NHDPlusID'] == r.properties['NHDPlusID']:
                
                id = r.properties['NHDPlusID']
                value_q = NHDPLusEROMMA[i+1]['properties']['QAMA']*cfs_to_ms
                discharge.append([('NHDPlusID',id), ('Discharge [m^3/s]',value_q)])
                
                value_v = NHDPLusEROMMA[i+1]['properties']['VAMA']*fs_to_ms
                velocity.append([('NHDPlusID',id), ('Velocity [m/s]',value_v)])
                
                length = r.properties['LengthKM']*km_to_m
                
                cross_area = value_q/value_v#FlowlineVAA[i+1]['properties']['TotDASqKm']
                                
                
                width = 1.0 #area/length*km_to_m
                height = cross_area * fs_to_ms #value_q/(value_v * width) 
                
                segment_length.append([('NHDPlusID',id), ('segment length [m]', length)])
                segment_depth.append([('NHDPlusID',id), ('segment depth [m]', height)])
                segment_width.append([('NHDPlusID',id), ('segment width [m]', width)])
                segment_crossA.append([('NHDPlusID',id), ('segment width [m]', cross_area * fs_to_ms)])
                
                # orientation
                #if Flowline[i+1]['properties']['NHDPlusID'] == r.properties['NHDPlusID']:
                p = Flowline[i+1]['geometry']['coordinates'][0][0]
                q = Flowline[i+1]['geometry']['coordinates'][0][-1] #note coord are in reverse order in NHD 
                print (k, r.properties['NHDPlusID'],length)
                n = get_normal([p,q])
                Normal.append([n[0],n[1],n[2]])
                
                # get number of cells and centroids
                
                
                
                #coordy = Flowline[i+1]['geometry']['coordinates']
                d4.append([('NHDPlusID',id), ('Discharge [m^3/s]',value_q),
                           ('Velocity [m/s]',value_v), ('segment length [m]', length),
                           ('segment depth [m]', height),
                           ('segment width [m]', width), ('orientation', [n[0],n[1],n[2]] )])
    
        
        for j in range(len(Flowline)):
            if Flowline[j+1]['properties']['NHDPlusID'] == r.properties['NHDPlusID']:
                id = r.properties['NHDPlusID']
                if (add_outlet_cell == True and k==0):
                    centroids, cell_lengths, cell_spacing, ncells = cell_lengths_and_centroids_simple(j+1,x_spacing_,outlet_cell=add_outlet_cell)
                else:
                    centroids, cell_lengths, cell_spacing, ncells = cell_lengths_and_centroids_simple(j+1,x_spacing_)
                d5.append([('NHDPlusID',id), ('centroids', centroids),
                           ('cell lengths [m]', cell_lengths), ('cell spacing [m]', cell_spacing),
                           ('number of cells', ncells)])
                #break
                
        
        for j in range(len(FlowlineVAA)):
            if FlowlineVAA[j+1]['properties']['NHDPlusID'] == r.properties['NHDPlusID']:
                id = r.properties['NHDPlusID']
                contrib_area =  FlowlineVAA[j+1]['properties']['TotDASqKm'] * km_to_m
                d6.append([('NHDPlusID',id), ('Contributing area', contrib_area)])
    
        #break
        
    return d4, d5, d6


In [15]:
x_spacing_=100
Data_NHD_ATS, DataMesh_NHD_ATS, DataArea_NHD_ATS  = River_Characteristics(x_spacing_)

0 25000400040729.0 2825.0
1 25000400108020.0 619.0
2 25000400040724.0 277.0
3 25000400007107.0 1970.0
4 25000400007106.0 819.0000000000001
5 25000400175089.0 1655.0
6 25000400074075.0 2900.0
7 25000400107798.0 1168.0000000000002
8 25000400108019.0 945.0000000000001
9 25000400175304.0 1092.0
10 25000400040725.0 1045.0000000000002
11 25000400007320.0 421.00000000000006
12 25000400074306.0 165.0
13 25000400007109.0 1291.0000000000002
14 25000400074076.0 987.0000000000001
15 25000400007108.0 765.0
16 25000400040325.0 1035.0000000000002
17 25000400141373.0 1173.0
18 25000400175091.0 787.0
19 25000400175090.0 775.0
20 25000400208871.0 871.0
21 25000400242778.0 1362.0


In [16]:
def Streams_connectivity():
    Node = []
    for r in river:
        for i in range(len(FlowlineVAA)):
            if FlowlineVAA[i+1]['properties']['NHDPlusID'] == r.properties['NHDPlusID']:
                id = r.properties['NHDPlusID']
                from_node = FlowlineVAA[i+1]['properties']['FromNode']
                to_node = FlowlineVAA[i+1]['properties']['ToNode']
                
                Node.append([('NHDPlusID',id), ('FromNode',from_node),('ToNode',to_node)])
    Nodes = dict()
    Nodes['connectivity'] = Node
    return Nodes

In [17]:
Connectivity = dict()
Connectivity_Orig = Streams_connectivity()

In [18]:
import networkx as nx
pairs = [(1, 2),(2, 3),(3, 4), (3, 5),(5, 9),(5, 7), (2, 8), (8,10), (8,11)]

graph_syn = nx.from_edgelist(pairs)
print (graph_syn.nodes)



[1, 2, 3, 4, 5, 9, 7, 8, 10, 11]


In [19]:
coweeta_pairs = []

NHD_Id_Index = []
for i,n in enumerate(Connectivity_Orig['connectivity']):
    NHD_Id_Index.append([n[0][1],i+1])
    coweeta_pairs.append((int(n[2][1]),int(n[1][1])))
Coweeta_Graph = nx.from_edgelist(coweeta_pairs)

coweeta_pairs_index = []
node_index_pair = []

for i,node in enumerate(Coweeta_Graph):
    node_index_pair.append((node,i+1))

def get_index_from_node(node):
    for n in node_index_pair:
        if node == n[0]:
            #print ('from node', node,n)
            break
    return n[1]

def get_node_from_index(index):
    for n in node_index_pair:
        if index == n[1]:
            break
    return n[0]

for n in Connectivity_Orig['connectivity']:
    x1 = get_index_from_node(int(n[2][1]))
    x2 = get_index_from_node(int(n[1][1]))
    coweeta_pairs_index.append((x1,x2))
    
Coweeta_Graph1 = nx.from_edgelist(coweeta_pairs_index)


In [20]:
def treeDiagnostic(graph):
    for node in graph:
        edges,_ = node_get_upstream_edges(graph,[node])
        for edge in edges:
            print (edge)
            assert edge[0] < edge[1]

def numNodes(graph):
    return len(graph.nodes)


def isNeighborNode(graph,node):
    ngh_nodes = list(graph.neighbors(node))
    return  True if len(ngh_nodes) > 0 else False

def node_get_edges(graph, node):
    
    edges = list(nx.edge_boundary(graph, node))
    return edges

def node_get_upstream_edges(graph, node):
    edges = list(nx.edge_boundary(graph, node))
    parent = -100
    for i,edge in enumerate(edges):
        if edge[0] > edge[1]:
            parent = edges[i]
            edges.pop(i)
    return edges, parent

def isNeighborBoundaryNode(graph, edge):
    ngh = list(graph.neighbors(edge[1]))
    return False if len(ngh) > 1 else True

def isBoundaryNode(graph, node):
    assert (node != 1)
    ngh = list(graph.neighbors(node))
    return False if len(ngh) > 1 else True

def get_node_index(graph, node):
    x = np.where(np.array(graph.nodes) == node)[0][0] + 1
    return x

def index_based_graph(graph):
    Nodes_new_ids = np.linspace(1,len(graph),len(graph), dtype='i')
    Nodes = graph.nodes

def implied_source_Raymond(Q, Contri_A,ncells, cell_area,mass_isconstant=False):
    #cell area is passed, and can be used if required
    g = 1020
    h = 0.345
    if (mass_isconstant):
        molar_mass = 1
    rho_molar = 1.0 #55555.55 # this is made one for chemistry since in ATS transport runs we use molar density 1 but for chemisty water density is 1000
    rho_water = 1000.0
    mass_molar = 1#rho_water / rho_molar  #[kg/moles]

    Qreach = Q[-1] # discharge at the first order stream outlet
    Source = np.zeros(len(Q)-1)
    #print ('Source: ',len(Source), len(cell_area), len(Q))
    for i in range(len(Q)-1):
        del_q = Q[i+1] - Q[i]
        
        C = np.round(g * (Qreach / (Contri_A * 8.64E+8) )**h,6)
        #print ('Qout=', Q[i+1], ' Qin = ',Q[i], ' dQ=',del_q, ' Cont=',C)
        Source[i] = ncells*(del_q * C) / mass_molar #/cell_area[i] # conver source [kg/sec] to [moles/sec]
       
        
    return Source

def implied_source(Q, cell_leng, cross_area, molar_mass=1):
    #cell area is passed, and can be used if required
    if (molar_mass > 1):
        rho_molar = 1.0 #55555.55 # this is made one for chemistry since in ATS transport runs we use molar density 1 but for chemisty water density is 1000
        rho_water = 1000.0
        molar_mass = rho_water / rho_molar  #[kg/moles]
    
    Source = np.zeros(len(Q)-1)
    #print ('Source: ',len(Source), len(cell_area), len(Q))
    for i in range(len(Q)-1):
        del_q = Q[i+1] - Q[i]
        Source[i] = del_q / (molar_mass * cell_leng[i]*cross_area) 
        # del_q [m^3/sec] * 1. [moles/m^3] = [moles/sec] ---> (*)
        # water source in ATS is in units of moles/(area*sec ), so we divide (*) by cell_area [m^2] too.
        #TODO: if Alquimia wrapper divides (which right now doesn't) by molar_density then we need to bring in molar
        #      density factor here too.
        
    return Source
    
    

In [21]:
1000/0.018

55555.55555555556

In [22]:
outpath = '/Users/ajc/Core/PreProcessData/Transport/2020/Coweeta_meshes_outletcell/'

In [42]:
from lxml import etree
import lxml.builder
import lxml.etree as ET

num_nodes = numNodes(Coweeta_Graph1)
fine_roots = []

Discharge =  []
CrossSectionArea = []
BoundaryCells = []
BoundaryReachCells = []
Total_cells = 0
#add_outlet_cell = True # fix sized cell at the outlet (adds one cell n) 

def WriteXMLNHD_Centroid(write=True):
    global Total_cells
    index = 0
    root = ET.Element("ParameterList", name="Main")
    ET.SubElement(root,"Parameter",name="infer cell centroids",type="bool", value="true")
    segments = ET.SubElement(root,"ParameterList", name="segments")
    #---- for regions
    root_region = ET.Element("ParameterList", name="Main")
    #--------
    nam = ''
    node_id_root = 0
    total_cells = 0
    bc_cell_id = 0
    tt = 0
    for i,node in enumerate(Coweeta_Graph1):
        #print (i)
        length = Data_NHD_ATS[i][3][1]
        width = 1#Data_NHD_ATS[i][5][1]
        orientation = (1,0,0)#Data_NHD_ATS[i][6][1]
        cross_area = Data_NHD_ATS[i][4][1]
        
        centroids = DataMesh_NHD_ATS[i][1][1]
        cell_lengths = DataMesh_NHD_ATS[i][2][1]
        ncells = DataMesh_NHD_ATS[i][4][1]
        
        total_cells = total_cells + ncells
        
        if i ==0:
            index = 0
            node_id_root = node
                       
            edges = node_get_edges(Coweeta_Graph1,[node])[0]
            
            nam = 'stream_' + str(edges[0]) + '_' + str(edges[1])
           
            streams = ET.SubElement(segments,"ParameterList", name="%s"%nam)
            ET.SubElement(streams, "Parameter", name="segment length [m]", type="double", value="%s"%length)
            
            ET.SubElement(streams,"Parameter",name="number of cells", type="int", value="%s"%ncells)
            
            n1,n2 = orientation[0],orientation[1]
            
            ET.SubElement(streams,"Parameter",name="orientation", type="Array(double)", value="{%s,%s,0}"%(n1,n2))
            ET.SubElement(streams,"Parameter",name="cross sectional area [m^2]", type="double", value="%s"%cross_area)
            ET.SubElement(streams,"Parameter",name="first tip type", type="string", value="boundary")
            ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="junction")
            ET.SubElement(streams,"Parameter",name="cell centroids", type="Array(double)", value="{%s}"%", ".join(map(str,centroids)))
            ET.SubElement(streams,"Parameter",name="cell lengths [m]", type="Array(double)", value="{%s}"% ", ".join(map(str,cell_lengths)))
            
            Discharge.append(Data_NHD_ATS[index][1][1])
            #print (Discharge)
            #break
            CrossSectionArea.append(Data_NHD_ATS[index][4][1])
            bc_cell_id = bc_cell_id + ncells
            tt  = tt + ncells
            r_ids = [-1 for i1 in range(ncells)] 
            BoundaryReachCells.append(r_ids)
            
            r_ids_first = [bc_cell_id -1 - i1 for i1 in range(ncells)]
            print (r_ids_first)
            reach_region = ET.SubElement(root_region,"ParameterList", name="%s"%nam)
            ET.SubElement(reach_region,"Parameter",name="entity", type="string", value="cell")
            ET.SubElement(reach_region,"Parameter",name="entity gids", type="Array(int)", value="{%s}"%", ".join(map(str,r_ids_first)))
            #print (ncells)
        else:
            edges, parent = node_get_upstream_edges(Coweeta_Graph1,[node])
           
            edge_len = len(edges)
            
            if edge_len == 0: #boundary edge
                continue
                
            for j, child_node in enumerate(edges):
                
                if child_node[1] != node_id_root:
                    index = child_node[1] - 2 
                    index_parent = child_node[0] - 2 
                    nam = 'stream_' + str(child_node[0]) + '_' + str(child_node[1])
                    streams = ET.SubElement(segments,"ParameterList", name="%s"%nam)
                   
                    centroids = DataMesh_NHD_ATS[index][1][1]
                    cell_lengths = DataMesh_NHD_ATS[index][2][1]
                    ncells = DataMesh_NHD_ATS[index][4][1]
                    cross_area = Data_NHD_ATS[index][4][1]
                    length = Data_NHD_ATS[index][3][1]
                    
                    ET.SubElement(streams,"Parameter",name="first tip type", type="string", value="branch")
                    ET.SubElement(streams,"Parameter",name="first tip branch segment", type="string", value="stream_%s_%s"%(parent[1],parent[0]))
                    ET.SubElement(streams,"Parameter",name="first tip branch segment tip", type="string", value="last")
                    
                    if isBoundaryNode(Coweeta_Graph1,child_node[1]):
                        ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="boundary")
                    else:
                        ET.SubElement(streams,"Parameter",name="last tip type", type="string", value="junction")
                        
                    n1,n2 = orientation[0],orientation[1]
                    
                    ET.SubElement(streams,"Parameter",name="orientation", type="Array(double)", value="{%s,%s,0}"%(n1,n2))
                    ET.SubElement(streams, "Parameter", name="segment length [m]", type="double", value="%s"%length)
                    ET.SubElement(streams,"Parameter",name="number of cells", type="int", value="%s"%ncells) 
                    ET.SubElement(streams,"Parameter",name="cross sectional area [m^2]", type="double", value="%s"%cross_area)
                    ET.SubElement(streams,"Parameter",name="cell centroids", type="Array(double)", value="{%s}"%", ".join(map(str,centroids)))
                    ET.SubElement(streams,"Parameter",name="cell lengths [m]", type="Array(double)", value="{%s}"% ", ".join(map(str,cell_lengths)))
                    
                    fine_roots.append(nam)
                    Discharge.append(Data_NHD_ATS[index][1][1])
                    CrossSectionArea.append(Data_NHD_ATS[index][4][1])
                    
                    bc_cell_id = bc_cell_id + ncells
                    # get boundary nodes ids
                    tt  = tt + ncells
                    #print (ncells)
                    if isBoundaryNode(Coweeta_Graph1,child_node[1]):
                        
                        BoundaryCells.append(bc_cell_id-1)
                        r_ids = [bc_cell_id -1 - i1 for i1 in range(ncells)] 
                        BoundaryReachCells.append(r_ids)
                        #print (nam, Discharge[-1],ncells,BoundaryReachCells[-1])
                        reach_region = ET.SubElement(root_region,"ParameterList", name="%s_1st_order"%nam)
                        ET.SubElement(reach_region,"Parameter",name="entity", type="string", value="cell")
                        ET.SubElement(reach_region,"Parameter",name="entity gids", type="Array(int)", value="{%s}"%", ".join(map(str,r_ids)))
                    else:
                        #BoundaryReachCells.append(r_ids)
                        #print (nam, Discharge[-1],ncells,BoundaryReachCells[-1])
                        r_ids = [bc_cell_id -1 - i1 for i1 in range(ncells)]
                        print ('N: ',ncells,r_ids)
                        reach_region = ET.SubElement(root_region,"ParameterList", name="%s"%nam)
                        ET.SubElement(reach_region,"Parameter",name="entity", type="string", value="cell")
                        ET.SubElement(reach_region,"Parameter",name="entity gids", type="Array(int)", value="{%s}"%", ".join(map(str,r_ids)))
                        
                        r_ids = [-1 for i1 in range(ncells)] 
                        BoundaryReachCells.append(r_ids)
                        
                    i = i + j
                #
            #
        #
        Total_cells = tt
        
        if (i+2 >= len(Coweeta_Graph1)):
            break
    sets = ET.SubElement(root,"ParameterList",name="sets",type="bool", value="true")
    ET.SubElement(sets,"Parameter",name="fine_root",type="Array(string)", value="{%s}"%", ".join(map(str,fine_roots)))
    Data= etree.tostring(root,encoding="unicode",pretty_print=True)
    Data_regions = etree.tostring(root_region,encoding="unicode",pretty_print=True)
    
    #outfile= open(outpath+ '/mesh_coweeta_mc-%sm.xml'%int(x_spacing_),'w') #mc for meshconvergence
    #outfile.write(Data)
    
    #write xml file for 1st order stream
    outfile= open(outpath + '/regions_fullcoweeta_mc-%sm.xml'%int(x_spacing_),'w') # reach length is the cell length
    outfile.write(Data_regions)




In [43]:
WriteXMLNHD_Centroid(write=True)

[29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
N:  7 [36, 35, 34, 33, 32, 31, 30]
N:  11 [47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37]
N:  3 [50, 49, 48]
N:  20 [80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61]
N:  30 [110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81]
N:  5 [164, 163, 162, 161, 160]
N:  2 [166, 165]
N:  8 [174, 173, 172, 171, 170, 169, 168, 167]
N:  10 [197, 196, 195, 194, 193, 192, 191, 190, 189, 188]
N:  8 [205, 204, 203, 202, 201, 200, 199, 198]


In [ ]:
#BoundaryCells
#BoundaryReachCells
#Total_cells

In [ ]:
0.061198907756493/len(BoundaryReachCells[8])
# S_i = dQ/N * Co, dQ = Qout - Qin, N=len(reach), Co = concentrate

In [ ]:
Discharge =  []
CrossSectionArea = []
BoundaryCells = []
ImpliedSourceTemp = [] #np.zeros((Total_cells))
ImpliedSource = np.zeros((Total_cells))
    
def Linearize_Discharge():
    QQ = []
    index = 0
    count = 0
    
    total_cells = 0
    map_id = []
    map_id.append(0)
    for i,node in enumerate(Coweeta_Graph1):
        if i ==-1:
            index = 0
            node_id_root = node
            Discharge.append(Data_NHD_ATS[index][1][1])
        else:
            edges, parent = node_get_upstream_edges(Coweeta_Graph1,[node])
            edge_len = len(edges)
            
            if (parent == -100):
                continue
            else:
                index_parent = parent[0] - 2
            
            #summing Qin
            #------------------------------------------------
            Qin = 0
            
            for edg in edges:
                index1 = edg[1]-2
                Qin = Qin + Data_NHD_ATS[index1][1][1] # Discharge
                
                
                count = count + 1
                map_id.append(index1)
            Boundary_Node = False
            if (edge_len==0): # boundary node
                
                Qin = 0
                Boundary_Node = True
                
                
            #------------------------------------------------
            Qout=Data_NHD_ATS[index_parent][1][1] # Dischage
            
            
            length = Data_NHD_ATS[index_parent][3][1] 
            centroids = DataMesh_NHD_ATS[index_parent][1][1]
            cell_spacing = np.round(DataMesh_NHD_ATS[index_parent][3][1],2)
            ncells = DataMesh_NHD_ATS[index_parent][4][1]
            cell_lengths = DataMesh_NHD_ATS[index_parent][2][1]
            
            
            m = (Qout - Qin)/length
            #print ('Spacing: ',cell_spacing)
            
            flux_f_orig = [round(Qin + m*x,5) for x in cell_spacing]
            #print (flux_f_orig)
            
            total_cells = total_cells + ncells
            # compute velocities
            cross_area = Data_NHD_ATS[index_parent][4][1]
            Velocity = Qout/(cross_area*cell_spacing[1])
                
            if not Boundary_Node:
                flux_f = flux_f_orig[1:]
                #print ('V: ',cross_area,Velocity)
            else:
                flux_f = flux_f_orig
                #print ('V-b: ',cross_area ,Velocity)
            
            #rr = all(np.diff(flux_f)==np.diff(flux_f)[0])
            #rrr = all(np.diff(cell_spacing)==np.diff(cell_spacing)[1])
            
            #print ('FLUX: ',flux_f)
            QQ.append(flux_f[::-1])
            #print (cell_lengths)
            cont_area = DataArea_NHD_ATS[index_parent][1][1]
            
            if Boundary_Node:
                #source = implied_source_Raymond(flux_f_orig,cont_area,ncells,cell_lengths,mass_isconstant=True)
                source = implied_source(flux_f_orig,cell_lengths,cross_area,molar_mass=1)
            else:
                source = implied_source(flux_f_orig,cell_lengths,cross_area,molar_mass=1)
                source = 1*source* 1 #this should be zero for first order streams implied source only
                #print ('Testing: ',source, flux_f_orig, cell_lengths)
            #print ('source ', index_parent, len(source),len(flux_f), flux_f[-1],flux_f[0],map_id[-1],)
            #print (Qout, Qin, length, ncells, len(flux_f), QQ)
            #print ('----------------------------------')
            ImpliedSourceTemp.append(source)
            #print (ncells, len(source))
            #print(source)
            #print('-------------------')
            #print ('HERE: ',ncells, source)
            
            #break    
    
    Q2 = []
    s11=0
    for i in range(0,len(QQ)): # to get the fluxes consistent with ATS faces
        Q2.append(QQ[map_id[i]])
        #print (len(QQ[map_id[i]]), map_id[i], QQ[map_id[i]][0],QQ[map_id[i]][-1])
        s11=s11+len(QQ[map_id[i]])
        
    full_watershed_implied_source = True
    first_order_reach = True
    
    if (full_watershed_implied_source):
        first_order_reach = False
    c = 0
    for i,d in enumerate(ImpliedSourceTemp):
        if (full_watershed_implied_source):
            
            #ImpliedSource = np.concatenate(ImpliedSourceTemp)
            
            for s in ImpliedSourceTemp[map_id[i]]:
                #print (c, map_id[i] ,s)
                ImpliedSource[c] =  s
                c = c +1
        elif (first_order_reach):
            for c,s in zip(BoundaryReachCells[i],ImpliedSourceTemp[map_id[i]]):
                ImpliedSource[c] =  s
                #print (i,c,s,map_id[i], len(ImpliedSourceTemp),len(BoundaryReachCells))
                if (not first_order_reach):
                    break # break if only injecting at the boundary cell
        elif (not first_order_reach):
            for c,s in zip(BoundaryReachCells[i],ImpliedSourceTemp[map_id[i]]):
                ImpliedSource[c] =  s
                #print (i,c,s,map_id[i], len(ImpliedSourceTemp),len(BoundaryReachCells))
          
                break # break if only injecting at the boundary cell
        
    print(ImpliedSource)
    np.set_printoptions(suppress=True)
    Q1 = np.concatenate(Q2,axis=0)
    #np.savetxt(outpath+ "/flux_interpolate-%sm.dat"%int(x_spacing_), Q1)
    np.savetxt(outpath + "/implied_source_full-%sm.dat"%int(x_spacing_), ImpliedSource)
    


In [ ]:
Linearize_Discharge()
#0.9080831404970909 0.8069423289574529 2825.0 15 15 # 200m

In [ ]:
k1 = (-0.80694 + 0.80784)#/25.l2 
k2= (-0.80784 + 0.80953)#/72.4138
print (k1,k2)
print (k1/25.12,k2/72.4138)

In [ ]:
Test200 = ImpliedSource

In [ ]:
BASE = ImpliedSource

In [ ]:
print (Test200)
print (BASE)

In [ ]:
x200=0.01435516*6
xbase = 0.08613093
print (x200,xbase)